# From Notebook to Kubeflow Pipeline using Lung Disease Detection

In this notebook, we will walk you through the steps of converting a machine learning model, which you may already have on a jupyter notebook, into a Kubeflow pipeline. As an example, we will make use of thelung diesease detection use case.

In this example we use:

* **Kubeflow pipelines** - [Kubeflow Pipelines](https://www.kubeflow.org/docs/pipelines/overview/pipelines-overview/) is a machine learning workflow platform that is helping data scientists and ML engineers tackle experimentation and productionization of ML workloads. It allows users to easily orchestrate scalable workloads using an SDK right from the comfort of a Jupyter Notebook.

**Note:** This notebook is to be run on a notebook server inside the Kubeflow environment. 

## Section 1: ML Origin Code

Take Lung Disease Diagnosis as aan example which used to detect multiple type of disease from X-ray images.
We chose Hernia model only with small datasets to demonstrate the funtionality of Kubeflow Pipelines without introducing too much complexity in the implementation of the ML model.


### 1.1 Install packages:

In [ ]:
# !python -m pip install --user --upgrade pip
# !pip install --user --upgrade pandas matplotlib numpy tensorflow keras scikit-learn h5py Pillow kfp pyyaml dvc dvc[ssh]

After the installation, we need to restart kernel for changes to take effect:

### 1.2 Import libraries

In [5]:
import os
import math
import warnings
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle, resample
from sklearn.metrics import  confusion_matrix, f1_score, roc_auc_score, roc_curve, auc, classification_report, accuracy_score, ConfusionMatrixDisplay
from sklearn.metrics import cohen_kappa_score

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.applications import VGG16, VGG19, ResNet101V2, ResNet50V2, InceptionV3, InceptionResNetV2, NASNetLarge, DenseNet121, DenseNet169, DenseNet201, Xception
from tensorflow.keras.optimizers import Nadam, SGD, RMSprop, Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.metrics import binary_accuracy, categorical_crossentropy
from tensorflow.keras.regularizers import l2
from tensorflow.keras.backend import clear_session
import sys
# sys.path.append('../data/') 
import argparse
import joblib
import json

### 1.2 Load Data

In [6]:
# load data
raw_csv_path = "../data/raw/Hernia_sample10/dataset.csv"
df = pd.read_csv(raw_csv_path, sep=",", encoding='utf-8')
df["image_index"]= df.image_index.apply(lambda x: x.replace('./datasets-registry', '../data/raw'))

processed_data_dir ="../data/processed/Hernia" 
try:
    os.makedirs(processed_data_dir, exist_ok=True)
    print("Directory '%s' created successfully" %processed_data_dir)
except OSError as error:
    print("Directory '%s' can not be created")

df.to_csv(os.path.join(processed_data_dir, "dataset.csv"), sep=",", index=False)
print(df.head())

Directory '../data/processed/Hernia' created successfully
                                         image_index  labels
0  ../data/raw/Hernia_sample10/positive/00009759_...  Hernia
1  ../data/raw/Hernia_sample10/positive/00021902_...  Hernia
2  ../data/raw/Hernia_sample10/positive/00014404_...  Hernia
3  ../data/raw/Hernia_sample10/positive/00012003_...  Hernia
4  ../data/raw/Hernia_sample10/positive/00000284_...  Hernia


### 1.3 Train And Evaluate

In [7]:
# split data
# Training data - used for training the model
# Validation data - used for tuning the hyperparameters and evaluate the models
# Test data - used to test the model after the model has gone through initial vetting by the validation set.
train_val_df, test_df = train_test_split(
    df,
    test_size = 0.2,
    random_state = 123,
    stratify=df['labels']
)
train_df, val_df = train_test_split(
    train_val_df,
    test_size = 0.2,
    random_state = 123,
    stratify=train_val_df['labels']
)

In [8]:
labels = train_df['labels'].unique()
labels = list(labels)

train_datagen = ImageDataGenerator(rescale=1./255.,
                                    rotation_range=20, 
                                    width_shift_range=0.2, 
                                    height_shift_range=0.2, 
                                    shear_range=0.3,
                                    zoom_range=0.3,
                                    horizontal_flip=True, 
                                    vertical_flip=False,
                                    fill_mode="nearest")

val_test_datagen = ImageDataGenerator(rescale=1./255.)

train_generator = train_datagen.flow_from_dataframe(dataframe=train_df,
                                                    #directory=IMG_PATH,
                                                    x_col='image_index',
                                                    y_col='labels',
                                                    target_size=(224, 224),
                                                    batch_size=32,
                                                    class_mode='binary',
                                                    seed = 42,
                                                    shuffle=True,
#                                                     classes=['0', 'Hernia'],
                                                    classes = labels,
                                                    interpolation='nearest')

val_generator = val_test_datagen.flow_from_dataframe(dataframe=val_df,
                                                        #directory=IMG_PATH,
                                                        x_col='image_index',
                                                        y_col='labels',
                                                        target_size=(224, 224),
                                                        batch_size=32,
                                                        class_mode='binary',
                                                        seed = 42,
                                                        classes = labels,
#                                                         classes=['0', 'Hernia'],
                                                        shuffle=True)
                                                    

test_generator = val_test_datagen.flow_from_dataframe(dataframe=test_df,
                                                        #directory=IMG_PATH,
                                                        x_col='image_index',
                                                        y_col='labels',
                                                        target_size=(224, 224),
                                                        batch_size=1,
                                                        class_mode='binary',
                                                        classes = labels,
#                                                         classes=['0', 'Hernia'],
                                                        shuffle=False)
print("#####################################")
print("indices", train_generator.class_indices)

Found 8 validated image filenames belonging to 2 classes.
Found 2 validated image filenames belonging to 2 classes.
Found 4 validated image filenames belonging to 2 classes.
#####################################
indices {'0': 0, 'Hernia': 1}


/opt/conda/lib/python3.7/site-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 4 invalid image filename(s) in x_col="image_index". These filename(s) will be ignored.
  .format(n_invalid, x_col)
/opt/conda/lib/python3.7/site-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 2 invalid image filename(s) in x_col="image_index". These filename(s) will be ignored.
  .format(n_invalid, x_col)


In [9]:
inceptresnet = InceptionResNetV2(
    weights='imagenet',
    input_shape=(224, 224, 3),
    include_top=False)

x = inceptresnet.output
x = GlobalAveragePooling2D(name="gap")(x)
x = Dense(256, activation='elu', kernel_initializer='he_uniform')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
pred = Dense(1, activation = "sigmoid", name="fc_out", kernel_initializer='he_uniform')(x)
model = Model(inputs=inceptresnet.input, outputs=pred)

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
model_dir = "../models/Hernia" 

try:
    os.makedirs(model_dir, exist_ok=True)
    print("Directory '%s' created successfully" %model_dir)
except OSError as error:
    print("Directory '%s' can not be created")

model_saved_path = os.path.join(model_dir, "model.h5")

checkpoint = ModelCheckpoint(model_saved_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1, mode='min')
                            
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min')

callbacks_list = [checkpoint, reduce_lr, earlyStopping]

model.fit(train_generator, 
        steps_per_epoch=math.ceil(train_generator.n/train_generator.batch_size),
        epochs=5,
        validation_data=val_generator,
        validation_steps=math.ceil(val_generator.n/val_generator.batch_size),
        callbacks=callbacks_list)

Directory '../models/Hernia' created successfully
Epoch 1/5
1/1 [==============================] - ETA: 0s - loss: 0.9198 - accuracy: 0.6250
Epoch 00001: val_loss improved from inf to 2.71308, saving model to ../models/Hernia/model.h5
1/1 [==============================] - 23s 23s/step - loss: 0.9198 - accuracy: 0.6250 - val_loss: 2.7131 - val_accuracy: 0.5000
Epoch 2/5
1/1 [==============================] - ETA: 0s - loss: 0.4303 - accuracy: 0.7500
Epoch 00002: val_loss did not improve from 2.71308
1/1 [==============================] - 1s 698ms/step - loss: 0.4303 - accuracy: 0.7500 - val_loss: 5.9716 - val_accuracy: 0.5000
Epoch 3/5
1/1 [==============================] - ETA: 0s - loss: 0.7544 - accuracy: 0.7500
Epoch 00003: val_loss did not improve from 2.71308
1/1 [==============================] - 1s 691ms/step - loss: 0.7544 - accuracy: 0.7500 - val_loss: 9.7235 - val_accuracy: 0.5000
Epoch 4/5
1/1 [==============================] - ETA: 0s - loss: 0.3876 - accuracy: 0.8750
Epoc

In [10]:
## predict
if not model:
    model = load_model(model_saved_path)
    test_generator.reset()
y_pred = model.predict(test_generator, steps=(math.ceil(test_generator.n/test_generator.batch_size)), verbose=1)
y_true = test_generator.classes

4/4 [==============================] - 1s 324ms/step


In [11]:
#evaluation measures
auc = roc_auc_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred >= 0.5)
acc = accuracy_score(y_true, y_pred >= 0.5)
#fpr, tpr, thresholds = roc_curve(y_true, y_pred)
#kappa is usually for imbalanced classes
kappa_score = cohen_kappa_score(y_true, y_pred >= 0.5)

cm = confusion_matrix(y_true, y_pred >= 0.5)    
#TN, FP, FN, TP = confusion_matrix(y_true, y_pred >= 0.5).ravel()
TN, FP, FN, TP = cm.ravel()
print (TN, FP, FN, TP)

#sensitivity or true positive rate
sensitivity = TP/(TP+FN)
#specificity or true negative rate
specificity = TN/(TN+FP)
#false positive rate
FPR = FP/(FP+TN)
#precision, positive predictive value
PPV = TP/(TP+FP)
#negative predictive value
NPV = TN/(TN+FN)

# print("InceptionResNetV2 model (weights=%f, img_w=%f, img_h=%f, channel=%f):" % (weights, img_w, img_h, channel))
print ('AUC: ', round(auc, 3))
print ('F1-score: ', round(f1, 3))
print ('Sensitivity: ', round(sensitivity, 3))
print ('Specificity: ', round(specificity, 3))
print ('False positive rate:', round(FPR, 3))
print ('PPV: ', round(PPV, 3))
print ('NPV: ', round(NPV, 3))
print ('Accuracy: ', round(acc, 3))
print ('Kappa Score: ', round(kappa_score, 3))

auc = auc.tolist()
f1 = f1.tolist()
acc = acc.tolist()
cm = cm.tolist()

report_dir = "../reports/Hernia" 
try:
    os.makedirs(report_dir, exist_ok=True)
    print("Directory '%s' created successfully" %report_dir)
except OSError as error:
    print("Directory '%s' can not be created")

score_file = os.path.join(report_dir, "scores.json")
with open(score_file, "w") as f:
    scores = {
        "auc": auc,
        "f1": f1,
        "cm": cm,
        "acc": acc
        }

    json.dump(scores, f, indent=4)

0 2 0 2
AUC:  0.5
F1-score:  0.667
Sensitivity:  1.0
Specificity:  0.0
False positive rate: 1.0
PPV:  0.5
NPV:  nan
Accuracy:  0.5
Kappa Score:  0.0
Directory '../reports/Hernia' created successfully


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in long_scalars


# Section 2: Kubeflow pipeline building

Nex step, we will make use of the containerized approach provided by Kubeflow to allow our model to be run using Kubernetes.

### 2.1 Install Kubeflow pipelines SDK

 The first step is to install the Kubeflow Pipelines SDK package.

In [12]:
# !pip install --user --upgrade kfp

After the installation, we need to restart kernel for changes to take effect:

Check if the install was successful:

In [2]:
# !which dsl-compile

You should see /usr/local/bin/dsl-compile above.

### 2.2 Build Container Components

The following cells define functions that will be transformed into lightweight container components. It is recommended to look at the corresponding Lung Disease Detection notebook to match what you see here to the original code.

In [3]:
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

<table>
  <tr><td>
    <img src="https://www.kubeflow.org/docs/images/pipelines-sdk-lightweight.svg"
         alt="Fashion MNIST sprite"  width="600">
  </td></tr>
  <tr><td align="center">
  </td></tr>
</table>

Import Kubeflow SDK

In [15]:
import kfp
import kfp.dsl as dsl
import kfp.components as comp

Create standalone python function - load_data()

In [16]:
def load_data():
    
     # func_to_container_op requires packages to be imported inside of the function.
    import os
    import paramiko
    import os
    from os import walk
    import pandas as pd
    
    ###### download datasets from ssh storage #######
    # 1 - Open a transport
    host="10.60.1.141"
    port = 22
    transport = paramiko.Transport((host, port))

    # 2 - Auth
    password="P@ssw0rd"
    username="user"
    transport.connect(username = username, password = password)

    # 3 - Go!
    sftp = paramiko.SSHClient()
    sftp._transport = transport
    sftp_download = sftp.open_sftp()
    # 4 - list all the files
    source_folder="datasets-registry/Hernia_sample10"
    cmd_line = 'find '+source_folder+' ' +'-type f'
    stdin, stdout, stderr = sftp.exec_command(cmd_line)
    test = stdout.read().decode("utf-8")
    test1 = test.splitlines()
    
    # 5 - Download the files and put in the local folder
    for file in test1:
        print(file)
        file_name = file.split('/')[-1]
        if file_name:
            base_path = file.split(file_name)[0]
            if not os.path.exists(base_path):
                os.makedirs(base_path, exist_ok=True)

            if file_name.split('.')[-1] in ('csv'):
                sftp_download.get(file, file)

    df = pd.read_csv('datasets-registry/Hernia_sample10/dataset.csv', sep=",", encoding='utf-8')
    print(df.head())
  
    

Create standalone python function - train_and_evaluate()

In [17]:
def train():
    # func_to_container_op requires packages to be imported inside of the function.
    import os
    import math
    import warnings
    import pandas as pd
    import numpy as np
    from sklearn.utils import shuffle, resample
    from sklearn.metrics import  confusion_matrix, f1_score, roc_auc_score, roc_curve, auc, classification_report, accuracy_score, ConfusionMatrixDisplay
    from sklearn.metrics import cohen_kappa_score

    import tensorflow as tf
    from tensorflow.keras.preprocessing.image import ImageDataGenerator
    from tensorflow.keras.models import Sequential, load_model, Model
    from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D, MaxPooling2D, BatchNormalization
    from tensorflow.keras.applications import VGG16, VGG19, ResNet101V2, ResNet50V2, InceptionV3, InceptionResNetV2, NASNetLarge, DenseNet121, DenseNet169, DenseNet201, Xception
    from tensorflow.keras.optimizers import Nadam, SGD, RMSprop, Adam
    from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
    from tensorflow.keras.metrics import binary_accuracy, categorical_crossentropy
    from tensorflow.keras.regularizers import l2
    from tensorflow.keras.backend import clear_session
    from sklearn.model_selection import train_test_split
    from sklearn.utils import shuffle, resample
    import sys
    import argparse
    import joblib
    import json
    
    #defining the install function
    import subprocess
    def install(name):
        subprocess.call(['pip', 'install', name])
    
    #install packages (installing numpy for the sake of demo)
    install('paramiko')
    import paramiko
    
    host="10.60.1.141"
    port = 22
    transport = paramiko.Transport((host, port))

    # 2 - Auth
    password="P@ssw0rd"
    username="user"
    transport.connect(username = username, password = password)

    # 3 - Go!
    sftp = paramiko.SSHClient()
    sftp._transport = transport
    sftp_download = sftp.open_sftp()
    # 4 - list all the files
    source_folder="datasets-registry/Hernia_sample10"
    cmd_line = 'find '+source_folder+' ' +'-type f'
    stdin, stdout, stderr = sftp.exec_command(cmd_line)
    test = stdout.read().decode("utf-8")
    test1 = test.splitlines()
    
    # 5 - Download the files and put in the local folder
    for file in test1:
        print(file)
        file_name = file.split('/')[-1]
        if file_name:
            base_path = file.split(file_name)[0]
            if not os.path.exists(base_path):
                os.makedirs(base_path, exist_ok=True)

            if file_name.split('.')[-1] in ('jpg', 'png'):
                sftp_download.get(file, file)
    
    df = pd.read_csv(f'{data_path}/dataset.csv', sep=",")
    print(df['labels'].value_counts())
    
    train_val_df, test_df = train_test_split(
        df,
        test_size = 0.2,
        random_state = 123,
        stratify=df['labels']
    )
    train_df, val_df = train_test_split(
        train_val_df,
        test_size = 0.2,
        random_state = 123,
        stratify=train_val_df['labels']
    )

    # when use v1
    # check whether need to do oversampling for positive datasets 
    pos_num = len(df[df["labels"] != '0'])
    neg_num = len(df[df["labels"] == '0'])
    print("positive:", pos_num, "negative:", neg_num)

    if pos_num+100 < neg_num:
        # do oversampling
        selected_df = pd.DataFrame(columns=['image_index', 'labels'])
        pos_sub_df = train_df[train_df["labels"] != '0']
        neg_sub_df = train_df[train_df["labels"] == '0']
        # print("negative size", len(neg_sub_df))
        pos_tem_df = resample(pos_sub_df,
                               replace=True,
                               n_samples=2500,
                               random_state=10)
        selected_df = selected_df.append(pos_tem_df[['image_index', 'labels']])
        
        
        neg_tem_df = resample(neg_sub_df,
                                   replace=True,
                                   n_samples=2500,
                                   random_state=10)
        selected_df = selected_df.append(neg_tem_df[['image_index', 'labels']])
        train_df = selected_df 
    
    labels = train_df['labels'].unique()
    labels = list(labels)
    
    train_datagen = ImageDataGenerator(rescale=1./255.,
                                    rotation_range=20, 
                                    width_shift_range=0.2, 
                                    height_shift_range=0.2, 
                                    shear_range=0.3,
                                    zoom_range=0.3,
                                    horizontal_flip=True, 
                                    vertical_flip=False,
                                    fill_mode="nearest")

    val_test_datagen = ImageDataGenerator(rescale=1./255.)

    train_generator = train_datagen.flow_from_dataframe(dataframe=train_df,
                                                        #directory=IMG_PATH,
                                                        x_col='image_index',
                                                        y_col='labels',
                                                        target_size=(224, 224),
                                                        batch_size=32,
                                                        class_mode='binary',
                                                        seed = 42,
                                                        shuffle=True,
                                                        # classes=['0', 'Hernia'],
                                                        classes = labels,
                                                        interpolation='nearest')

    val_generator = val_test_datagen.flow_from_dataframe(dataframe=val_df,
                                                         #directory=IMG_PATH,
                                                         x_col='image_index',
                                                         y_col='labels',
                                                         target_size=(224, 224),
                                                         batch_size=32,
                                                         class_mode='binary',
                                                         seed = 42,
                                                         classes = labels,
                                                        #  classes=['0', 'Hernia'],
                                                         shuffle=True
                                                         )

    test_generator = val_test_datagen.flow_from_dataframe(dataframe=test_df,
                                                          #directory=IMG_PATH,
                                                          x_col='image_index',
                                                          y_col='labels',
                                                          target_size=(224, 224),
                                                          batch_size=1,
                                                          class_mode='binary',
                                                          classes = labels,
                                                        #   classes=['0', 'Hernia'],
                                                          shuffle=False)
    print("#####################################")
    print("indices", train_generator.class_indices)

    inceptresnet = InceptionResNetV2(
        weights='imagenet',
        input_shape=(224, 224, 3),
        include_top=False)

    x = inceptresnet.output
    x = GlobalAveragePooling2D(name="gap")(x)
    x = Dense(256, activation='elu', kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    pred = Dense(1, activation = "sigmoid", name="fc_out", kernel_initializer='he_uniform')(x)
    model = Model(inputs=inceptresnet.input, outputs=pred)

    optimizer = Adam(lr=0.001)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    
    # Define model saved path 
    model_dir = "../models/Hernia" 
    try:
        os.makedirs(model_dir, exist_ok=True)
        print("Directory '%s' created successfully" %model_dir)
    except OSError as error:
        print("Directory '%s' can not be created")

    model_saved_path = os.path.join(model_dir, "model.h5")

    checkpoint = ModelCheckpoint(model_saved_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1, mode='min')
                                
    earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min')

    callbacks_list = [checkpoint, reduce_lr, earlyStopping]

    model.fit(train_generator, 
            steps_per_epoch=math.ceil(train_generator.n/train_generator.batch_size),
            epochs=5,
            validation_data=val_generator,
            validation_steps=math.ceil(val_generator.n/val_generator.batch_size),
            callbacks=callbacks_list)
    
    ## predict
    if not model:
        model = load_model(model_saved_path)
        test_generator.reset()
    y_pred = model.predict(test_generator, steps=(math.ceil(test_generator.n/test_generator.batch_size)), verbose=1)
    y_true = test_generator.classes

    #evaluation measures
    auc = roc_auc_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred >= 0.5)
    acc = accuracy_score(y_true, y_pred >= 0.5)
    #fpr, tpr, thresholds = roc_curve(y_true, y_pred)
    #kappa is usually for imbalanced classes
    kappa_score = cohen_kappa_score(y_true, y_pred >= 0.5)
    
    cm = confusion_matrix(y_true, y_pred >= 0.5)    
    #TN, FP, FN, TP = confusion_matrix(y_true, y_pred >= 0.5).ravel()
    TN, FP, FN, TP = cm.ravel()
    print (TN, FP, FN, TP)
    
    #sensitivity or true positive rate
    sensitivity = TP/(TP+FN)
    #specificity or true negative rate
    specificity = TN/(TN+FP)
    #false positive rate
    FPR = FP/(FP+TN)
    #precision, positive predictive value
    PPV = TP/(TP+FP)
    #negative predictive value
    NPV = TN/(TN+FN)

    # print("InceptionResNetV2 model (weights=%f, img_w=%f, img_h=%f, channel=%f):" % (weights, img_w, img_h, channel))
    print ('AUC: ', round(auc, 3))
    print ('F1-score: ', round(f1, 3))
    print ('Sensitivity: ', round(sensitivity, 3))
    print ('Specificity: ', round(specificity, 3))
    print ('False positive rate:', round(FPR, 3))
    print ('PPV: ', round(PPV, 3))
    print ('NPV: ', round(NPV, 3))
    print ('Accuracy: ', round(acc, 3))
    print ('Kappa Score: ', round(kappa_score, 3))
    
    auc = auc.tolist()
    f1 = f1.tolist()
    acc = acc.tolist()
    cm = cm.tolist()
    
    # export to scores.json
    report_dir = "../reports/Hernia" 
    try:
        os.makedirs(report_dir, exist_ok=True)
        print("Directory '%s' created successfully" %report_dir)
    except OSError as error:
        print("Directory '%s' can not be created")

    score_file = os.path.join(report_dir, "scores.json")
    with open(score_file, "w") as f:
        scores = {
            "auc": auc,
            "f1": f1,
            "cm": cm,
            "acc": acc
            }

        json.dump(scores, f, indent=4)

Create train and predict lightweight components, converting functions to container operation

### 2.3 Build Kubeflow Pipeline

Our next step will be to create the various components that will make up the pipeline. Define the pipeline using the *@dsl.pipeline* decorator.

The pipeline function is defined and includes a number of paramters that will be fed into our various components throughout execution. Kubeflow Pipelines are created decalaratively. This means that the code is not run until the pipeline is compiled. 

A [Persistent Volume Claim](https://kubernetes.io/docs/concepts/storage/persistent-volumes/) can be quickly created using the [VolumeOp](https://) method to save and persist data between the components. Note that while this is a great method to use locally, you could also use a cloud bucket for your persistent storage.

Define the pipeline and define parameters to be fed into pipeline

### 2.4 Run pipeline

Finally we feed our pipeline definition into the compiler and run it as an experiment. This will give us 2 links at the bottom that we can follow to the [Kubeflow Pipelines UI](https://www.kubeflow.org/docs/pipelines/overview/pipelines-overview/) where you can check logs, artifacts, inputs/outputs, and visually see the progress of your pipeline.

Define some environment variables which are to be used as inputs at various points in the pipeline.

Create a client to enable communication with the Pipelines API server.

Compile and Run the pipeline

/opt/conda/lib/python3.7/site-packages/kfp/dsl/_container_op.py:1039: FutureWarning: Please create reusable components instead of constructing ContainerOp instances directly. Reusable components are shareable, portable and have compatibility and support guarantees. Please see the documentation: https://www.kubeflow.org/docs/pipelines/sdk/component-development/#writing-your-component-definition-file The components can be created manually (or, in case of python, using kfp.components.create_component_from_func or func_to_container_op) and then loaded using kfp.components.load_component_from_file, load_component_from_uri or load_component_from_text: https://kubeflow-pipelines.readthedocs.io/en/stable/source/kfp.components.html#kfp.components.load_component_from_file
  category=FutureWarning,
